<a href="https://colab.research.google.com/github/Danielmarques-ch/Controle-de-Pedidos/blob/main/Controle_de_Pedidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime
import smtplib
from email.message import EmailMessage

# ================= CONFIG EMAIL =================

EMAIL_REMETENTE = "SEU E-MAIL"
SENHA_APP = "SUA SENHA API"
EMAIL_DESTINO = "SEU E-MAIL"

# ==============================================

def enviar_email(produto, fornecedor, prazo):

    msg = EmailMessage()
    msg["Subject"] = "⚠ ALERTA DE PRAZO DE ENTREGA"
    msg["From"] = EMAIL_REMETENTE
    msg["To"] = EMAIL_DESTINO

    msg.set_content(f"""
FIQUE ATENTO!

Produto: {produto}
Fornecedor: {fornecedor}
Prazo: {prazo}

Esse pedido ultrapassa 15 dias de entrega.
""")

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(EMAIL_REMETENTE, SENHA_APP)
        smtp.send_message(msg)

# =================================================

escopo = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

credenciais = Credentials.from_service_account_file(
    "credenciais.json",
    scopes=escopo
)

cliente = gspread.authorize(credenciais)
planilha = cliente.open("controle_pedidos")
aba = planilha.sheet1

st.title("📦 Sistema de Controle de Pedidos")

contrato = st.text_input("Contrato")
fornecedor = st.text_input("Fornecedor")
produto = st.text_input("Produto")

valor_produto = st.number_input(
    "Valor do Produto (R$)",
    min_value=0.0,
    format="%.2f"
)

prazo = st.text_input("Prazo entrega (dd/mm/aaaa)")

if st.button("Cadastrar pedido"):

    if contrato and fornecedor and produto and prazo:

        hoje = datetime.today()

        try:
            data_prazo = datetime.strptime(prazo, "%d/%m/%Y")
            dias = (data_prazo - hoje).days

            if dias > 15:
                status = "FIQUE ATENTO"
                enviar_email(produto, fornecedor, prazo)
            else:
                status = "EM ANDAMENTO"

            aba.append_row([
                contrato,
                fornecedor,
                produto,
                valor_produto,
                hoje.strftime("%d/%m/%Y"),
                prazo,
                status
            ])

            st.success(f"Pedido cadastrado — Status: {status}")

        except:
            st.error("Formato de data inválido")

    else:
        st.warning("Preencha todos os campos")
